In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.chrome.options import Options
import json
import re

In [55]:
cases_links = []
all_span_tuples = []

total_pages = 2  # alterar para a quantidade correta

for page in range(1, total_pages + 1):
    url = f'https://radiopaedia.org/search?lang=us&page={page}&scope=cases&sort=date_of_publication'
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    elements = soup.find_all('a', class_='search-result-case')

    divs = soup.select('#search-results div.col-xs-12.search-results-listing a div.search-result-body div.search-result-description div.search-result-modalities')

    for div, element in zip(divs, elements):
        spans = div.find_all('span')
        span_texts = [span.text.strip() for span in spans]
        all_span_tuples.append(tuple(span_texts))

        if 'CT' not in span_texts and 'MRI' not in span_texts:
            link = element.get('href')
            cases_links.append('https://radiopaedia.org' + link)

In [3]:
options = Options()
options.headless = True  # Ativado para que o navegador não seja aberto.

driver = webdriver.Chrome(options=options)

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re
import json

for case in cases_links:
    print(case)

    try:
        driver.get(case)

        # Espera até que o elemento 'fa-clickable' seja clicável
        div_element = driver.find_element(By.CLASS_NAME, 'fa-clickable')
        driver.execute_script("arguments[0].click();", div_element)
        print(div_element)

        # Espera até que o script seja carregado na página
        script_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//script[contains(text(), "var stackedImages")]'))
        )

        # Obtenha o texto do script
        codigo_js = script_element.get_attribute('text')

        match = re.search(r'var stackedImages = (.*?);', codigo_js)

        if match:
            json_data = match.group(1)

            data = json.loads(json_data)
            image_info = []

            for entry in data:
                for image in entry['images']:
                    info = {
                        'public_filename': image['public_filename'],
                        'plane_projection': image['plane_projection']
                    }
                    image_info.append(info)

            for info in image_info:
                print(f"Public Filename: {info['public_filename']}")
                print(f"Plane Projection: {info['plane_projection']}")
                print('-' * 30)
        else:
            print("Pattern not found.")

    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error: {e}")

    finally:
        # Aguarde um tempo antes de prosseguir para evitar problemas de temporização
        time.sleep(2)

        # Atualize a página
        driver.refresh()

https://radiopaedia.org/cases/schatzker-type-v-tibial-plateau-fracture-with-head-of-fibula-fracture-7?lang=us
<selenium.webdriver.remote.webelement.WebElement (session="15f788dd33e54d3d0acccf8023e07fdb", element="972CCE0DAB289755482872AAB453278D_element_521")>
Public Filename: https://prod-images-static.radiopaedia.org/images/64056188/3bf9445a137d94a6d3189589bc3828fbe3a1777c25f221f3e1c708ed74fc98e2.png
Plane Projection: Frontal
------------------------------
Public Filename: https://prod-images-static.radiopaedia.org/images/64056187/0859cc09468e6b7636913fd80bf9939ed633ffa1e0b1e013696d110d59af07a7.png
Plane Projection: Lateral
------------------------------
https://radiopaedia.org/cases/usual-interstitial-pneumonitis-4?lang=us
<selenium.webdriver.remote.webelement.WebElement (session="15f788dd33e54d3d0acccf8023e07fdb", element="F74A7F9409E21989380C5DDF3AC229C3_element_593")>
Public Filename: https://prod-images-static.radiopaedia.org/images/64137731/bdc563758d3f0ad424e286da5b6f73b6c4c92

In [60]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def extrair_dados(url):
    dados = {'URL': [], 'Texto': []}

    try:
        response = requests.get(url, headers={'Content-Type': 'text/html; charset=utf-8'})
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
        return
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
        return
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
        return
    except requests.exceptions.RequestException as err:
        print("Oops! Something went wrong:", err)
        return

    dados['URL'].append(url)

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    divs = soup.find_all('div', class_='case-section')

    for div_element in divs:
        p_elements = div_element.find_all('p')

        for p_element in p_elements:
            texto = p_element.get_text(strip=True)
            print('Texto dentro do parágrafo:', texto)
            dados['Texto'].append(texto)

        # Caso contrário, procure por dados na div
        dados_items = div_element.find_all('div', class_='data-item')
        for item in dados_items:
            label = item.find('strong', class_='data-item-label').get_text(strip=True)
            valor = item.get_text(strip=True, separator=' ').replace(label, '', 1)
            dados_texto = f'{label}: {valor}'
            print(dados_texto)
            dados['Texto'].append(dados_texto)

    if not dados['Texto']:
        dados['Texto'].append('No data found')

    # Convert 'Texto' list to a single string
    dados['Texto'] = ', '.join(dados['Texto'])

    return dados

# Initialize an empty DataFrame
final_df = pd.DataFrame({'URL': [], 'Texto': []})

# Loop through each URL and append the extracted data to the DataFrame
for url in cases_links:
    extracted_data = extrair_dados(url)
    temp_df = pd.DataFrame(extracted_data)
    final_df = pd.concat([final_df, temp_df], ignore_index=True)

# Save the final DataFrame to a CSV file
csv_filename = 'output.csv'
final_df.to_csv(csv_filename, index=False)
print(f'DataFrame saved to {csv_filename}')


Texto dentro do parágrafo: Alleged motorbike skidded, fell and hit the left lower limb directly towards the road.
Age::  35 years
Gender::  Male
Texto dentro do parágrafo: Bicondylar fracture of the tibial plateau with fracture of the head of the fibula.
Texto dentro do parágrafo: The Schatzker classification is used to classify tibial plateau fractures according to the configuration of the fracture. This case shows a Schatzker type V tibial plateau fracture along with a fracture of the head of the fibula.
Texto dentro do parágrafo: Cough short of breath 6 month
Age::  60 years
Gender::  Male
Texto dentro do parágrafo: There is pulmonary fibrosis with ample subpleural interlobular septal thickening and peripheral honeycombing most pronounced in the bases.
Texto dentro do parágrafo: On imaging, usual interstitial pneumonia usually presents with a lung volume loss and an apicobasal gradient of peripheral septal thickening, bronchiectasis, and honeycombing.
Texto dentro do parágrafo: Chie

NameError: name 'df' is not defined